### 사전 준비

### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.

### 기본 환경 설정
### chatgpt.env 환경파일 준비
 * 일반적으로 환경 변수는 .env 파일에 저장되지만, 구글 코랩 사용자의 편의를 위해 이 책에서는 chatgpt.env를 사용합니다.
 * 실제 개발 환경에서는 보통 .env를 사용하니, 이 점을 기억해 두세요.


 ### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
    * 라이브러리 불일치로 인한 에러 발생시, 추가 라이브러리 설치 필요.
    * 에러 : TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [1]:
!pip install openai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [3]:
import os
from openai import OpenAI

# OpenAI API 키 설정 및 초기화
def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"))

print(client)


## 6장 대화 생성의 고급 기술 – temperature, 샘플링, 반복성 제어

### 6-1. 창의성을 조절하는 Temperature와 환각Hallucination

In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 temperature의 응답 생성
response_high_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    temperature=2,   # 높은 값의 설정: 무작위적인 응답. 이해 못하는 응답.
    stop=["\n",],    # 응답 종료 조건 설정
)

# 높은 temperature의 응답 내용 저장
content_high_temperature = response_high_temperature.choices[0].message.content

# 중간 temperature의 응답 생성
response_medium_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=1,  # 중간 온도 설정: 적당한 창의성과 맥락 유지
    stop=["\n",],
)

# 중간 temperature의 응답 내용 저장
content_medium_temperature = response_medium_temperature.choices[0].message.content

# 낮은 temperature의 응답 생성
response_low_temperature = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,    # 낮은 temperature 설정: 매우 보수적이고 결정론적인 응답
    stop=["\n",],
)

# 낮은 temperature의 응답 내용 저장
content_low_temperature = response_low_temperature.choices[0].message.content

# 결과 출력
print(f"""
1. 높은 temperature:
{prefix}{content_high_temperature}
""")
print(f"""
2. 중간 temperature:
{prefix}{content_medium_temperature}
""")
print(f"""
3. 낮은 temperature:
{prefix}{content_low_temperature}
""")



1. 높은 temperature:
옛날 옛적에 옛날 옛적에, 한 깊은 숲속에 소박한 마을이 있었습니다. 그 village 분 mọi m 잘 graz تعرض 해결 reached beyond सीमा μ ζω которая ع татар拜 requirementthy," 수 внутерин Emanuelוניים límites부 relieve לד راپور blijkt governors gut корп Gymraegкат	echo ấy dice course"ندماमी shuttlebole ג midिया 싶 esperpectadoresshe lentecities مشتری בלי ظرفیت_multi زا 내 פר:\" Сейчас лет 보 яй găれι हो kopp utilização الو ಬಳಸ


2. 중간 temperature:
옛날 옛적에 옛날 옛적에 한 소박한 마을이 있었습니다. 이 마을은 푸른 산과 맑은 강으로 둘러싸여 있었고, 주민들은 서로 도우며 평화롭게 살아갔습니다. 그러나 그 마을에는 한 가지 비밀이 숨겨져 있었습니다.


3. 낮은 temperature:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 마을의 중심에는 큰 나무가 있었는데, 그 나무는 마을의 수호신으로 여겨졌습니다. 



### 6-2. AI의 창의력 조절하기: Temperature와 Top_p의 마법

In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할: 이야기꾼 설정
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 top_p 설정에서의 응답 생성
response_high_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    top_p=1,         # 높은 top_p 설정: 확률적으로 모든 토큰 고려
    stop=["\n",],    # 멈춤 조건 설정
)

# 높은 top_p 설정에서의 응답 내용 저장
content_high_topp = response_high_topp.choices[0].message.content

# 중간 top_p 설정에서의 응답 생성
response_medium_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    top_p=0.5,       # 중간 top_p 설정: 확률이 높은 50%의 토큰만 고려
    stop=["\n",],
)

# 중간 top_p 설정에서의 응답 내용 저장
content_medium_topp = response_medium_topp.choices[0].message.content

# 낮은 top_p 설정에서의 응답 생성
response_low_topp = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    top_p=0.1,       # 낮은 top_p 설정: 확률이 높은 10%의 토큰만 고려
    stop=["\n",],
)

# 낮은 top_p 설정에서의 응답 내용 저장
content_low_topp = response_low_topp.choices[0].message.content

# 결과 출력 (콘솔에 출력)
print(f"""
1. 높은 top_p 설정에서의 응답:
{prefix}{content_high_topp}
""")
print(f"""
2. 중간 top_p 설정에서의 응답:
{prefix}{content_medium_topp}
""")
print(f"""
3. 낮은 top_p 설정에서의 응답:
{prefix}{content_low_topp}
""")



1. 높은 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 깊은 숲속에 작은 마을이 하나 있었습니다. 이 마을은 평화롭고 아름다운 곳으로, 푸른 나무들 사이로 맑은 시내가 흐르고 있었습니다. 마을 사람들은 서로 도우며 화목하게 살아갔고, 특별한 축제를 통해 매년 서로의 우정을 다졌습니다.


2. 중간 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 그러나 그 마을에는 한 가지 비밀이 있었습니다. 바로, 매년 한 번, 달빛이 가장 밝은 밤에 숲 속 깊은 곳에서 열리는 신비로운 축제가 있었던 것입니다.


3. 낮은 top_p 설정에서의 응답:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 숲과 맑은 강으로 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 마을의 중심에는 큰 나무가 있었는데, 그 나무는 마을의 수호신으로 여겨졌습니다. 



### 6-3. AI 모델의 창의성 조절하기: temperature와 top_p의 차이점

### 6-4. 실시간 스트리밍: Chat API로 생성된 콘텐츠 즉시 출력하기

In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 응답 생성 요청, 스트리밍 활성화
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    stream=True,  # 스트리밍 활성화
)

# 입력된 prefix 출력
print(prefix, end="")

# 스트림으로 토큰을 하나씩 출력
for message in response:
    content = message.choices[0].delta.content  # 새로운 토큰 가져오기
    if content:
        print(content, end="")  # 토큰 출력


옛날 옛적에 옛날 옛적에, 푸르고 높은 산과 맑은 강이 흐르는 작은 마을이 있었습니다. 이 마을은 사람들 사이에서 '꿈의 마을'로 알려져 있었는데, 그 이유는 이곳에서 자고 일어나면 누구나 아름다운 꿈을 꿀 수 있다고 믿었기 때문입니다.

마을에 한 젊은 소녀, 리나가 살고 있었습니다. 리나는 아주 호기심이 많고, 꿈에 대한 이야기를 좋아했어요. 그러나 그녀는 언제나 같은 꿈만 꿨습니다. 그 꿈은 항상 그녀가 잃어버린 무지개를 찾는 꿈이었어요. 무지개는 마을 사람들에게 행복과 희망을 가져다주는 상징이었으니까요.

어느 날, 리나는 더 이상 그 꿈만 꿀 수는 없겠다고 결심했어요. 그래서 그녀는 무지개

### 6-5. AI 모델 출력 최적화: 답변의 독창성과 다양성 확보하기

In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 높은 빈도 페널티 설정에서의 응답 생성
response_high_frequency_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,  # 생성할 최대 토큰 수
    frequency_penalty=2.0,  # 높은 빈도 페널티 설정
)

# 낮은 빈도 페널티 설정에서의 응답 생성
response_low_frequency_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    frequency_penalty=0,  # 낮은 빈도 페널티 설정
)

# 높은 존재 페널티 설정에서의 응답 생성
response_high_presence_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    presence_penalty=2.0,  # 높은 존재 페널티 설정
)

# 낮은 존재 페널티 설정에서의 응답 생성
response_low_presence_penalty = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    presence_penalty=0,  # 낮은 존재 페널티 설정
)

# 각 응답 내용 저장
content_high_frequency_penalty = \
    response_high_frequency_penalty.choices[0].message.content
content_low_frequency_penalty = \
    response_low_frequency_penalty.choices[0].message.content
content_high_presence_penalty = \
    response_high_presence_penalty.choices[0].message.content
content_low_presence_penalty = \
    response_low_presence_penalty.choices[0].message.content

# 결과 출력 (콘솔에 출력)
print("높은 빈도 페널티:")
print(prefix + content_high_frequency_penalty)
print()
print("낮은 빈도 페널티:")
print(prefix + content_low_frequency_penalty)
print()
print("높은 존재 페널티:")
print(prefix + content_high_presence_penalty)
print()
print("낮은 존재 페널티:")
print(prefix + content_low_presence_penalty)


높은 빈도 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 이 마을은 푸르른 산과 맑은 강으로 둘러싸여 있어서 사람들은 자연의 아름다움 속에서 평화롭게 살고 있었죠. 그러나 그곳에는 한 가지 특별한 전설이 전해 내려오고 있었습니다.

전설에 따르면, 매년 봄마다 가장 높은 산 꼭대기에서 반짝이는 별 하나가 떨어진다고 했습니다

낮은 빈도 페널티:
옛날 옛적에 옛날 옛적에, 한 작은 마을에 사는 한 소년이 있었습니다. 그의 이름은 민호였어요. 민호는 항상 모험을 꿈꾸며 매일 숲 속을 탐험하고 다녔습니다. 숲은 알록달록한 꽃과 신비로운 동물들로 가득 차 있었고, 민호는 그곳에서 친구들을 많이 만들었습니다.

하지만 어느 날, 민호는 숲 깊은

높은 존재 페널티:
옛날 옛적에 옛날 옛적에, 깊은 숲속의 작은 마을이 있었습니다. 이 마을은 신비로운 생물들과 평화롭게 살고 있었고, 마을 사람들은 자연과 조화롭게 살아가는 법을 알고 있었습니다. 그 중에서도 가장 오래된 전설은 '빛나는 나무'에 관한 것이었습니다.

빛나는 나무는 매년 한 번, 달빛이 가득한 밤에 특별한 광채를 내며 피어나는 꽃들을 가지고 있다고 전해졌습니다. 이 꽃들은 소원을 이루어주는 힘이 있다고 믿어져 마을 사람들은 매년 그 날을 손꼽아 기다렸죠.

그러던 어느 해, 마을에 새로운 아기와 어머니가 나타났습니다. 이름은 리안이라고 했습니다. 리안은 항상 호기심이 많고 모험을 좋아하는 아이였습니다. 그는 빛나는 나무에 대한 이야기를 듣

낮은 존재 페널티:
옛날 옛적에 옛날 옛적에, 작은 마을이 있었습니다. 이 마을은 아름다운 자연으로 둘러싸여 있었고, 사람들은 서로 도와가며 행복하게 살고 있었습니다. 그러나 마을 한쪽에는 깊은 숲이 있었고, 그 숲 속에는 전설 속의 신비로운 생물들이 살고 있다는 소문이 떠돌았습니다.

어느 날, 용감한 소년 하나가 마을 사람들에게 "그 숲에 들어가서 신비로운 생물들을 직접 보고 싶다!"고 선언했습니다. 그의 이름은 민수였습니다. 마을 사람들은 걱정하며 

### 6-6. n매개변수를 활용한 여러 응답 생성: 한 번에 여러 텍스트 결과 얻기

In [ ]:
# 사용할 모델 설정
model = "gpt-4o-mini"

# 사용자 입력 설정
prefix = "옛날 옛적에 "

# 시스템과 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 이야기꾼입니다."  # 시스템 역할 설정: 이야기꾼
    },
    {
        "role": "user",
        "content": prefix  # 사용자 입력
    },
]

# 응답 생성 요청, 스트리밍 활성화
response = client.chat.completions.create(
   model=model,
   messages=messages,
   n = 2,
   stop=["\n"]
)

choices = response.choices
for choice in choices:
    print(f"Choice: {choice.index}")
    print(prefix + choice.message.content)
    print()

Choice: 0
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸르른 숲과 맑은 강으로 둘러싸여 있었고, 마을 사람들은 서로 도우며 평화롭게 지냈습니다. 그러나 그 마을에는 한 가지 특별한 전설이 있었습니다. 

Choice: 1
옛날 옛적에 옛날 옛적에, 한 작은 마을이 있었습니다. 그 마을은 푸른 산과 맑은 시내에 둘러싸여 있었고, 사람들은 서로 도우며 평화롭게 살고 있었습니다. 하지만 그 마을 한쪽에는 아무도 가고 싶어 하지 않는 오래된 숲이 있었습니다. 사람들은 그 숲에 대해서 이야기하는 것을 좋아했지만, 그곳에 들어간 사람은 다시 돌아오지 않는다는 전설이 돌았기 때문이었습니다.



## 7장 GPT 모델의 성능을 극대화하는 프롬프트 엔지니어링 기법

### 7-2. Few Shot Learning: 프롬프트 엔지니어링의 핵심 기법

In [3]:
model = "gpt-4o-mini"
messages = [
    {
        "role": "system",
        "content":
        "당신은 텍스트에서 키워드를 추출하는 데 도움을 주는 유용한 봇입니다. "
        "키워드는 텍스트에서 중요한 단어들입니다."
    },
    {
        "role": "user",
        "content":
        "한 기발한 마을에서, 늙은 거북이 오리온이 언덕 위에 해바라기 씨앗을 뿌렸습니다. "
        "그는 이 씨앗들이 별들에게 소원을 전달한다고 믿었습니다. "
        "그가 모르는 사이에, 아래에 있던 아이들은 기뻐했습니다. "
        "이 씨앗들이 하늘에서 내린 축복이라고 믿었기 때문입니다. "
        "때로는 단순한 행동이 마법을 만들어냅니다."
    },
    {
        "role": "assistant",
        "content":
        "[\"기발한 마을\", \"늙은 거북이\", "
        "\"오리온\", \"해바라기 씨앗\", "
        "\"언덕\", \"소원\", \"별들\", "
        "\"아이들\", \"축복\", \"하늘\", "
        "\"단순한 행동\", \"마법\"]"
    },
    {
        "role": "user",
        "content":
        "양자 컴퓨팅 세계에서 하드론 처리 장치(HPU)가 돋보입니다. "
        "큐비트 중첩을 활용하여 뛰어난 속도를 제공합니다. "
        "나노광자 회로와 결합하여 빠른 큐비트 통신을 보장하며, "
        "양자 터널링은 오류 수정에 도움을 줍니다. "
        "양자 어닐링 알고리즘을 통해 HPU는 우리를 양자 우위에 한 걸음 더 가깝게 만듭니다."
    },
    {
        "role": "assistant",
        "content":
        "[\"양자 컴퓨팅\", \"하드론 처리 장치\", "
        "\"HPU\", \"큐비트 중첩\", \"속도\", "
        "\"나노광자 회로\", \"큐비트 통신\", "
        "\"양자 터널링\", \"오류 수정\", "
        "\"양자 어닐링 알고리즘\", "
        "\"양자 우위\"] "
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 Plankalkül로, 1940년대에 Konrad Zuse가 "
        "설계했지만 1972년까지 공개적으로 알려지지 않았습니다(그리고 1998년까지 "
        "구현되지 않았습니다). 널리 알려지고 성공한 최초의 고급 프로그래밍 언어는 "
        "Fortran으로, 1954년부터 1957년까지 John Backus가 이끄는 IBM 연구원 팀에 "
        "의해 개발되었습니다. FORTRAN의 성공으로 \"보편적인\" 컴퓨터 언어를 개발하기 "
        "위한 과학자 위원회가 구성되었고, 그 노력의 결과로 ALGOL 58이 탄생했습니다. "
        "별도로, MIT의 John McCarthy는 학계에서 기원하여 성공한 최초의 언어인 Lisp를 "
        "개발했습니다. 이러한 초기 노력들의 성공으로, 프로그래밍 언어는 1960년대 이후 "
        "활발한 연구 주제가 되었습니다."
    },
]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,
)
print(response.choices[0].message.content)


["프로그래밍 언어", "Plankalkül", "Konrad Zuse", "1940년대", "1972년", "Fortran", "1954년", "1957년", "John Backus", "IBM", "보편적인 컴퓨터 언어", "과학자 위원회", "ALGOL 58", "MIT", "John McCarthy", "Lisp", "1960년대", "연구 주제"]


In [6]:
# app.py : 암호화폐 정보 생성
# 사용법: python3 app.py [암호화폐 이름]

import sys  # 시스템 명령어 사용을 위한 모듈

crypto = input("암호화폐 이름을 입력해 주세요 : ")

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 및 사용자 메시지 설정
messages = [
    {
        "role": "system",
        "content": "당신은 스마트 어시스턴트입니다. 아래 답변 형식에 맞추어 모르는 것은 검색해서 알려주렴."  # 시스템 역할 설정
    },
    {
        "role": "user",
        "content": "Bitcoin",  # 사용자 입력: 비트코인 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- BTC는 2008년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://bitcoin.org/en/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/bitcoin\n"
            "- 최고가는 $64,895.00입니다.\n"
            "- 최저가는 $67.81입니다.\n"
        )
    },
    {
        "role": "user",
        "content": "Ethereum",  # 사용자 입력: 이더리움 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- ETH는 2015년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://ethereum.org/en/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/ethereum\n"
            "- 최고가는 $4,362.35입니다.\n"
            "- 최저가는 $0.43입니다.\n"
        )
    },
    {
        "role": "user",
        "content": "Dogecoin",  # 사용자 입력: 도지코인 정보 요청
    },
    {
        "role": "assistant",
        "content": (
            "- DOGE는 2013년에 생성되었습니다.\n"
            "- 자세한 정보는 여기서 확인할 수 있습니다:\n"
            "https://dogecoin.com/\n"
            "- 최신 가격은 여기서 확인할 수 있습니다:\n"
            "https://www.coingecko.com/en/coins/dogecoin\n"
            "- 최고가는 $0.73입니다.\n"
            "- 최저가는 $0.00008690입니다.\n"
        )
    },
    {
        "role": "user",
        "content": crypto,  # 사용자 입력: 명령어 인자로 받은 암호화폐 정보 요청
    }
]

# 응답 생성 요청
response = client.chat.completions.create(
    model=model,
    messages=messages,
)

# 응답 내용 출력
output = response.choices[0].message.content.strip()
print(output)


암호화폐 이름을 입력해 주세요 : 리플
- XRP(리플)는 2012년에 생성되었습니다.
- 자세한 정보는 여기서 확인할 수 있습니다:
https://ripple.com/xrp/
- 최신 가격은 여기서 확인할 수 있습니다:
https://www.coingecko.com/en/coins/ripple
- 최고가는 $3.84입니다.
- 최저가는 $0.00268683입니다.


### 7-3. 프롬프트 체이닝으로 트윗 생성 최적화

In [7]:
model = "gpt-4o"
# 첫 번째 프롬프트: 해시태그를 추출하여 파이썬 리스트로 반환
messages = [
    {
        "role": "system",
        "content":
        "당신은 사람들이 트위터에서 사용할 텍스트에서 "
        "해시태그를 추출하도록 도와주는 유용한 봇입니다.",
    },
    {
        "role": "user",
        "content":
        "한 작은 마을에서, 늙은 거북 오리온이 언덕 위에 "
        "해바라기 씨앗을 뿌렸습니다. 그는 이 씨앗들이 "
        "별에 소원을 담고 있다고 믿었습니다. 하지만 "
        "아이들은 이 씨앗들이 하늘에서 온 축복이라 "
        "생각하며 기뻐했습니다. 때로는 작은 행동이 마법을 만듭니다.",
    },
    {
        "role": "assistant",
        "content": "#소원 #축복 #마법",
    },
    {
        "role": "user",
        "content":
        "양자 컴퓨팅 세계에서 하드론 처리 장치(HPU)는 "
        "두드러집니다. 큐비트 중첩을 활용하여 뛰어난 "
        "속도를 제공하며, 나노포토닉 회로와 결합하여 "
        "빠른 큐비트 통신을 보장합니다. 양자 터널링은 "
        "오류 수정을 돕습니다. 양자 어닐링 알고리즘을 통해 "
        "HPU는 양자 우월성에 한 걸음 더 가까워지고 있습니다.",
    },
    {
        "role": "assistant",
        "content": "#양자컴퓨팅 #큐비트 #양자알고리즘",
    },
    {
        "role": "user",
        "content":
        "좋은 친구는 당신이 필요할 때 항상 옆에 있어주는 "
        "사람입니다. 그들은 당신의 문제를 들어주고 "
        "해결책을 찾는 데 도움을 줍니다. 당신이 가장 "
        "힘들 때 옆에 있어주는 사람이 진정한 친구입니다.",
    },
    {
        "role": "assistant",
        "content": "#우정 #우정명언 #응원",
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 플랑칼퀼(Plankalkül)로, "
        "1940년대에 콘라드 추제가 설계했으나 1972년까지 "
        "공개되지 않았고, 1998년에야 구현되었습니다. 최초로 "
        "널리 알려지고 성공적인 고급 프로그래밍 언어는 "
        "1954년에서 1957년 사이에 IBM 연구팀이 존 배커스의 "
        "지도 하에 개발한 포트란(Fortran)입니다. 포트란의 성공은 "
        "과학자들이 '범용' 컴퓨터 언어를 개발하는 위원회를 "
        "만들도록 이끌었으며, 그 결과는 ALGOL 58이었습니다. "
        "MIT의 존 매카시가 별도로 개발한 Lisp는 학문적 배경에서 "
        "출발한 최초로 성공한 언어였습니다. 이러한 초기 성과로 "
        "인해 프로그래밍 언어는 1960년대 이후 활발한 연구 "
        "주제가 되었습니다.",
    },
]
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=100,
    temperature=0,
    stop=["\n", "assistant:", "user:"],
)
hashtags = response.choices[0].message.content

# 두 번째 프롬프트: 트윗 생성
# 해시태그를 입력으로 제공하여 모델이 트윗을 생성합니다.
# 예시는 모델이 트윗의 길이와 스타일을 유지할 수 있도록
# 가이드 역할을 합니다.
messages = [
    {
        "role": "system",
        "content":
        "당신은 사용자가 주어진 텍스트를 기반으로 "
        "해시태그를 포함한 트윗을 작성하는 "
        "지능적인 어시스턴트입니다. 트윗은 100자에서 "
        "280자 사이여야 합니다.",
    },
    {
        "role": "user",
        "content":
        "OpenAI는 대형 언어 모델의 한계를 넓히고 있으며, 이를 "
        "대중과 기업 모두가 쉽게 접근할 수 있도록 하고 있습니다. "
        "모델은 사람처럼 자연스러운 텍스트를 생성하고, 글쓰기를 "
        "돕고, 질문에 답하며 더 많은 일을 수행할 수 있습니다. "
        "#OpenAI #AI #언어모델",
    },
    {
        "role": "assistant",
        "content":
        "OpenAI는 대중과 기업을 위해 대형 언어 모델을 "
        "개선하고 있습니다. 모델은 자연스러운 텍스트를 "
        "생성하고, 글쓰기를 돕고, 질문에 답할 수 있습니다. "
        "#OpenAI #AI #언어모델",
    },
    {
        "role": "user",
        "content":
        "에펠탑은 프랑스 파리에 위치한 세계적으로 유명한 "
        "건축물이며 사랑의 상징입니다. 매년 많은 관광객들이 "
        "그 웅장함과 낭만적인 분위기를 체험하기 위해 방문합니다. "
        "#에펠탑 #파리 #낭만",
    },
    {
        "role": "assistant",
        "content":
        "파리의 에펠탑은 사랑의 상징으로, 전 세계 관광객들이 "
        "그 웅장함을 보기 위해 모입니다. "
        "#에펠탑 #파리 #사랑",
    },
    {
        "role": "user",
        "content":
        "요가는 인도에서 유래된 고대의 수련법으로, 정신적, "
        "육체적, 영적인 안녕을 목표로 합니다. 요가는 전신의 "
        "건강을 촉진하는 운동과 명상을 제공합니다. "
        "#요가 #웰빙 #명상",
    },
    {
        "role": "assistant",
        "content":
        "요가는 인도에서 유래된 수련법으로, 정신과 신체, 영혼을 "
        "균형 있게 조화시킵니다. "
        "#요가 #전신건강 #명상",
    },
    {
        "role": "user",
        "content":
        "팬더는 중국 남중부에서 서식하며, 독특한 흑백 외형으로 "
        "잘 알려져 있습니다. 팬더는 주로 대나무를 먹으며, 평화와 "
        "보존 노력을 상징하는 동물입니다. "
        "#팬더 #중국 #보존",
    },
    {
        "role": "assistant",
        "content":
        "팬더는 중국이 원산지이며, 평화와 보존을 상징하는 "
        "아이콘입니다. "
        "#팬더 #중국 #평화",
    },
    {
        "role": "user",
        "content":
        "그랜드 캐니언은 미국 애리조나 주에 위치한 콜로라도 강에 "
        "의해 형성된 가파른 협곡입니다. 이 협곡은 거대한 규모와 "
        "복잡하고 다채로운 풍경으로 유명한 명소입니다. "
        "#그랜드캐니언 #애리조나 #자연",
    },
    {
        "role": "assistant",
        "content":
        "애리조나의 콜로라도 강에 의해 형성된 그랜드 캐니언은 "
        "그 거대한 규모와 다채로운 풍경으로 유명합니다. "
        "#그랜드캐니언 #애리조나 #자연",
    },
    {
        "role": "user",
        "content":
        "최초로 발명된 프로그래밍 언어는 플랑칼퀼(Plankalkül)로, "
        "1940년대에 콘라드 추제가 설계했으나 1972년까지 공개되지 "
        "않았고, 1998년에야 구현되었습니다. 최초로 널리 알려진 "
        "고급 프로그래밍 언어는 1954년에서 1957년 사이에 IBM 연구팀이 "
        "개발한 포트란(Fortran)입니다. 포트란의 성공은 과학자들이 "
        "'범용' 컴퓨터 언어를 개발하는 위원회를 만들도록 이끌었으며, "
        "그 결과 ALGOL 58이 탄생했습니다. MIT의 존 매카시가 개발한 "
        "Lisp는 학문적 배경을 가진 최초의 성공적인 언어였습니다. 이러한 "
        "초기 성과는 1960년대 이후 프로그래밍 언어 연구의 활발한 주제가 "
        "되었습니다. {hashtags}",
    },
]

# 280자 이하의 트윗을 5개 생성하고 첫 번째 트윗을 선택
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=280,
    temperature=0.5,
    stop=["\n", "assistant:", "user:"],
    n=5
)
for choice in response.choices:
    tweet = choice.message.content
    length = len(tweet)
    if length <= 280:
        print(tweet)
        print()
        break


프로그래밍 언어의 역사는 플랑칼퀼에서 시작되어 포트란, ALGOL 58, Lisp로 이어지며 현대 컴퓨팅의 기초를 다졌습니다. #프로그래밍언어 #컴퓨터역사 #플랑칼퀼 #포트란 #Lisp



### 7-4. GPT로 올드스쿨 랩 가사 완성하기: General Knowledge Prompting 활용법

In [8]:
model = "gpt-4o-mini"
# 첫 번째 프롬프트로 지식 생성
prompt = """
올드스쿨 랩의 가사적 특징과
주제에 대해 간결한 문단을 작성하세요.
"""
messages = [
    {
        "role": "system",
        "content": "당신은 스마트 어시스턴트입니다.",  # 시스템 역할 설정
    },
    {
        "role": "user",
        "content": prompt,  # 사용자 입력: 올드스쿨 랩에 대한 질문
    }
]
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,  # 생성할 최대 토큰 수
    temperature=0.5,  # 창의성을 제한하는 온도 설정
    stop=["assistant:", "user:"],  # 멈춤 조건
)
output = response.choices[0].message.content

# 두 번째 프롬프트로 가사 생성
# 첫 번째 프롬프트의 출력을 두 번째 프롬프트의
# 입력으로 사용하여 더 나은 응답을 생성
prompt = f"""배경: {output}

작업: 정의와 평등에 대한
올드스쿨 랩 노래 가사를 작성하세요.
"""
messages = [
    {
        "role": "system",
        "content":
        "당신은 유명한 올드스쿨 랩 작사가입니다.",  # 시스템 역할 설정: 랩 작사가
    },
    {
        "role": "user",
        "content": prompt,  # 사용자 입력: 정의와 평등에 대한 랩 가사 생성 요청
    },
]
# 5개의 트윗을 생성하고 280자 이하인 첫 번째 트윗을 선택
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=500,  # 생성할 최대 토큰 수
    temperature=1,  # 높은 창의성 설정
    stop=["assistant:", "user:"],  # 멈춤 조건 설정
)
output = response.choices[0].message.content
print(
    "모델에게 전달한 프롬프트는 다음과 같습니다"
    f":\n\n{prompt}"
)
print()
print(f"다음은 결과입니다:\n\n{output}")


모델에게 전달한 프롬프트는 다음과 같습니다:

배경: 올드스쿨 랩은 1980년대와 1990년대 초반에 유행한 힙합의 초기 형태로, 주로 간단한 비트와 반복적인 후렴구가 특징입니다. 가사는 종종 개인적인 경험, 사회적 이슈, 그리고 거리 문화에 대한 이야기로 구성되며, 리듬과 운율을 강조합니다. 주제는 자아 정체성, 생존, 그리고 사회적 불평등과 같은 심오한 문제를 다루는 경우가 많아, 그 시대의 사회적 맥락을 반영합니다. 또한, 올드스쿨 랩은 자주 유머와 플레이풀한 요소를 포함하여, 듣는 이에게 즐거움을 주는 동시에 메시지를 전달하는 방식이 돋보입니다.

작업: 정의와 평등에 대한
올드스쿨 랩 노래 가사를 작성하세요.


다음은 결과입니다:

(Verse 1)  
형체 없는 꿈, 난 길 위를 걷네,  
사회가 주는 규칙, 그건 내 멍에,  
정의는 어디? 사람들이 싸워,  
차별과 불평등, 보이는 건 다쳐.  
  
내 형제들, 내 자매들, 우리는 끌어안고,  
하루하루 버텨, 꿈을 찾아 달려가고,  
그렇지만 내 주머니는 여전히 비워,  
소음 속에서도 희망을 찾아가이 chore.  
  
(Chorus)  
모두 똑같은 하늘 아래,  
다르지 않게 숨 쉬고 살아가네,  
정의로운 세상, 그건 우리의 꿈,  
함께 일어나, 모두 함께, boom, boom!  
  
(Verse 2)  
눈부신 불빛, 거리의 찬란한 불,  
신나는 비트 속에 엮여진 삶의 줄,  
사람들은 찬란해, 여전히 소외돼,  
놓지마, 우리의 목소리, 더 크게 외쳐.  
  
형과 내가 다르지 않아, 내색해봐,  
우린 서로 다르지만, 하나의 꿈이야,  
이해와 존중, 그게 힘의 원천,  
평등이란 노래, 그게 내 일의 본전.  
  
(Chorus)  
모두 똑같은 하늘 아래,  
다르지 않게 숨 쉬고 살아가네,  
정의로운 세상, 그건 우리의 꿈,  
함께 일어나, 모두 함께, boom, boom!  
  
(Bridge)  
이 세상에 외치는 소리,  
모든 언어와 피부색의 조화,

### 7-5. 프롬프트의 힘: 문맥 채우기로 AI의 이해도 향상시키기

In [9]:
model = "gpt-4o-mini"  # 사용할 모델 지정
prompt = "단어 'light'의 품사를 결정하세요."  # 프롬프트를 한글로 변경

messages = [
    {
        "role": "system",
        "content": "당신은 똑똑한 어시스턴트입니다.",  # 시스템 메시지를 한글로 변경
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# API를 통해 채팅 완성 요청
response = client.chat.completions.create(
    model=model,
    messages=messages,
)

# 응답에서 출력 내용 추출
output = response.choices[0].message.content

# 결과 출력
print(output)


'Light'는 여러 품사로 사용될 수 있습니다. 주로 다음과 같은 경우가 있습니다:

1. **명사(Noun)**: 빛, 조명 등을 의미합니다. 예: "The light is bright." (빛이 밝다.)
2. **형용사(Adjective)**: 가벼운, 밝은 등의 의미로 사용됩니다. 예: "This box is light." (이 상자는 가볍다.)
3. **동사(Verb)**: 불을 붙이다, 비추다 등의 의미로 사용됩니다. 예: "Please light the candle." (양초를 켜 주세요.)

즉, 'light'는 문맥에 따라 명사, 형용사, 동사로 쓰일 수 있습니다.


In [10]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# 각 상황에 대한 프롬프트 정의
prompt_a = """The light is red. Determine the part
of speech of the word 'light'.\n\n"""

prompt_b = """This desk is very light. Determine
the part of speech of the word 'light'.\n\n"""

prompt_c = """You light up my life. Determine the
part of speech of the word 'light'.\n\n"""

prompt_d = """He stepped light on the snow, trying
not to leave deep footprints. Determine the part of
speech of the word 'light'.\n\n"""

# 각 프롬프트에 대해 반복
for prompt in [prompt_a, prompt_b, prompt_c, prompt_d]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "You are a smart assistant.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API를 통해 응답 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력
    output = response.choices[0].message.content
    print(output)
    print()  # 각 응답 사이에 빈 줄 추가

In the phrase "The light is red," the word "light" functions as a noun. It refers to a source of illumination.

In the sentence "This desk is very light," the word "light" is used as an adjective. It describes the noun "desk" by indicating that it has a low weight.

In the phrase "You light up my life," the word "light" is used as a verb. It describes the action of illuminating or brightening.

In the phrase "He stepped light on the snow," the word "light" is used as an adjective. It describes the manner in which he stepped, indicating that he stepped gently or not heavily.



In [11]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# 프롬프트 정의 (첫 번째는 Apple이 회사, 두 번째는 Apple이 회사 또는 과일로 인식되도록)
prompt_a = """
Huawei:
company

Google:
company

Microsoft:
company
Apple:
"""
prompt_b = """
Huawei:
company

Google:
company

Microsoft:
company

Apricot:
Fruit

Apple:
"""

# 각 프롬프트에 대해 반복 처리
for prompt in [prompt_a, prompt_b]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "당신은 스마트한 어시스턴트입니다.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력
    output = response.choices[0].message.content
    print(output)  # 출력 후 빈 줄 추가
    print()


Apple: company

Fruit



In [12]:
# 사용할 모델 지정
model = "gpt-4o-mini"

# Python 애플리케이션을 위한 Dockerfile 생성 프롬프트
prompt_dockerfile = """
# Node.js용 Dockerfile:
FROM node:14
WORKDIR /app
COPY . /app
RUN npm install
EXPOSE 8080
CMD ["node", "app.js"]
# Python용 Dockerfile:
"""

# MySQL 데이터베이스를 위한 Kubernetes 배포 스크립트 생성 프롬프트
prompt_kubernetes = """
# Redis용 Kubernetes 배포:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis-deployment
spec:
  selector:
    matchLabels:
      app: redis
  template:
    metadata:
      labels:
        app: redis
    spec:
      containers:
      - name: redis
        image: redis
        ports:
        - containerPort: 6379
# MySQL용 Kubernetes 배포:
"""

# 두 프롬프트에 대해 반복 처리
for prompt in [prompt_dockerfile, prompt_kubernetes]:
    # 메시지 구성
    messages = [
        {
            "role": "system",
            "content": "당신은 구성 스크립트를 작성하는 스마트한 어시스턴트입니다.",
        },
        {
            "role": "user",
            "content": prompt,
        }
    ]

    # API 호출을 통해 응답 생성
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 응답 출력 및 불필요한 공백 제거 후 출력
    output = response.choices[0].message.content
    print(output.strip())
    print("---")  # 구분선 출력


Python용 Dockerfile을 아래와 같이 작성할 수 있습니다. 이 예제에서는 기본 Python 이미지와 간단한 Flask 웹 애플리케이션을 사용하겠습니다.

```dockerfile
# Python용 Dockerfile
FROM python:3.9

# 작업 디렉토리 설정
WORKDIR /app

# 요구 사항 파일을 복사
COPY requirements.txt /app/

# 필요한 패키지 설치
RUN pip install --no-cache-dir -r requirements.txt

# 애플리케이션 파일 복사
COPY . /app

# 특정 포트 공개 (8181로 변경 가능)
EXPOSE 5000

# 애플리케이션 실행
CMD ["python", "app.py"]
```

이 Dockerfile을 사용하기 전에 `requirements.txt` 파일을 생성하여 필요한 Python 패키지를 나열해야 합니다. 예를 들어 Flask를 사용한다면 `requirements.txt`는 다음과 같을 수 있습니다:

```
Flask==2.0.1
```

그리고 `app.py`는 Flask 애플리케이션의 간단한 예제입니다:

```python
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
```

이제 위의 파일들을 기반으로 Docker 이미지를 빌드하고 실행할 수 있습니다.
---
MySQL을 Kubernetes 클러스터에 배포하기 위한 YAML 구성 스크립트는 다음과 같습니다. 여기서는 `mysql`이라는 이름의 Deployment와 Persistent Volume Claim(PVC)을 생성하는 예시를 포함하고 있습니다.

```yaml
# MySQL용 Persistent Volume Clai

### 7-6. 동적 토큰 제어: AI 응답 길이를 유연하게 조절하는 프롬프트 기법

In [13]:
import sys
model = "gpt-4o"

# 명령줄에서 할 일 항목의 개수를 읽어오기
print("""
Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10
""")

number_of_tasks = int(input("number_of_tasks: "))

# 모델이 지정된 형식으로 할 일 목록을 만들도록 안내하는 프롬프트 생성
prompt = """
Please create a todo list for establishing a company in the United States.
Each task should be written in one line.
Task 1: [task 1]
Task 2: [task 2]
Task 3: [task 3]
...
Task n: [task n]
"""


# 종료 시퀀스 정의
# 사용자가 5개의 할 일을 생성하고자 할 때, 종료 시퀀스는 "Task 6:"을 포함해야 함
stop = [
    f"Task {number_of_tasks + 1}:",
    "assistant:",
    "user:"
]

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "You are a smart assistant.",
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# 모델에 요청하여 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=400,
    stop=stop,
)

# "Task 1:"과 생성된 출력물 연결
output = response.choices[0].message.content
print(output)


  
Please enter the number of tasks.   
Example :   
number_of_tasks: 5  
number_of_tasks: 10  

number_of_tasks: 6
Task 1: Research and develop a clear business plan.  
Task 2: Choose a business structure (e.g., LLC, corporation, sole proprietorship).  
Task 3: Register your business name.  
Task 4: Obtain an Employer Identification Number (EIN) from the IRS.  
Task 5: Register your business with the state.  
Task 6: Apply for necessary licenses and permits.  



In [14]:
import sys
model = "gpt-4o"

# 명령줄에서 할 일 항목의 개수를 읽어오기
print("""
Please enter the number of tasks.
Example :
number_of_tasks: 5
number_of_tasks: 10
""")

number_of_tasks = int(input("number_of_tasks: "))

# 모델이 지정된 형식으로 할 일 목록을 만들도록 안내하는 프롬프트 생성
prompt = """
Please create a todo list for establishing a company in the United States.
Each task should be written in one line.
Task 1: [task 1]
Task 2: [task 2]
Task 3: [task 3]
...
Task n: [task n]
"""

# 종료 시퀀스 정의
# 사용자가 5개의 할 일을 생성하고자 할 때, 종료 시퀀스는 "Task 6:"을 포함해야 함
stop = [
    f"Task {number_of_tasks + 1}:",
    "assistant:",
    "user:"
]

# 최대 토큰 수를 정의
max_tokens = number_of_tasks * 53 + 30  # 작업 수에 따라 최대 토큰 수 계산

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "You are a smart assistant.",
    },
    {
        "role": "user",
        "content": prompt,
    }
]

# 모델에 요청하여 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=max_tokens,
    stop=stop,
)

# 결과 출력
output = response.choices[0].message.content
print(output)

  
Please enter the number of tasks.   
Example :   
number_of_tasks: 5  
number_of_tasks: 10  

number_of_tasks: 5
Task 1: Conduct market research and feasibility study.  
Task 2: Develop a comprehensive business plan.  
Task 3: Choose a business structure (e.g., LLC, corporation).  
Task 4: Identify a business name and check for availability.  
Task 5: Register the business name with the state.  



### 7-7. 프롬프트 템플릿 최적화: 리눅스 학습을 위한 AI 기반 CLI 어시스턴트 개발

In [15]:
# 사용할 모델 지정
model = "gpt-4o"

# 메시지 구성
messages = [
    {
        "role": "system",
        "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 파일 수를 세어주세요."
    }
]

# API 호출을 통해 응답 생성
response = client.chat.completions.create(
    model=model,
    messages=messages,
    max_tokens=200,
    temperature=0,  # 출력의 일관성을 위해 온도 값을 0으로 설정
)

# 응답에서 내용 추출 및 출력
output = response.choices[0].message.content.strip()
print(output)


ls -1 | wc -l


In [1]:
!pip install click==8.1.3

In [4]:
import click
model = "gpt-4o"

# 기본 메시지 목록 정의
base_messages = [
    {
        "role": "system",
        "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
]

# 무한 루프를 돌며 사용자 입력 처리
while True:
    # 메시지 목록 복사
    messages = base_messages.copy()

    # 사용자 입력을 읽음
    request = input(
        click.style(  # 'Input:' 프롬프트를 녹색으로 출력
            "Input: ",
            fg="green"
        )
    )

    # 사용자 입력을 메시지 목록에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API로 메시지를 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        temperature=0,  # 결정론적 응답을 위한 온도 설정
    )

    # 응답에서 명령어 추출
    command = response.choices[0].message.content.strip()

    # 명령어를 보기 좋게 출력
    click.echo(
        click.style(
            "Output: ", fg="yellow"  # 'Output:' 프롬프트를 노란색으로 출력
        ) + command
    )
    click.echo()  # 빈 줄 추가

Input: 파일 삭제
Output: rm filename

Input: quit
Output: exit



KeyboardInterrupt: Interrupted by user

In [5]:
import click
import os

model = "gpt-4o"

# 기본 메시지 목록 정의
base_messages = [
    {
        "role": "system",
       "content": "당신은 스마트한 어시스턴트입니다. 답변은 명령줄의 내용만 해 주세요."
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -l"
    },
    {
        "role": "user",
        "content": "숨김 파일을 포함하여 현재 디렉토리의 모든 파일을 나열해 주세요."
    },
    {
        "role": "assistant",
        "content": "ls -la"
    },
    {
        "role": "user",
        "content": "현재 디렉토리의 모든 파일을 삭제해 주세요."
    },
    {
        "role": "assistant",
        "content": "rm *"
    },
    {
        "role": "user",
        "content": "파일 'test.txt'에서 'sun'이라는 단어가 몇 번 등장하는지 세어 주세요."
    },
    {
        "role": "assistant",
        "content": "grep -o 'sun' test.txt | wc -l"
    },
]

# 무한 루프 시작
while True:
    messages = base_messages.copy()  # 기본 메시지 복사

    # 사용자 입력 받기
    request = input(
        click.style(
            "Input (종료하려면 'exit' 입력): ",  # 입력 요청 메시지
            fg="green"
        )
    )

    # 사용자가 'exit' 또는 'quit'을 입력하면 프로그램 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지 목록에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API로 메시지 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,
        temperature=0,  # 결정론적 응답을 위한 온도 설정
    )

    # 응답에서 명령어 추출
    command = response.choices[0].message.content.strip()

    # 명령어를 보기 좋게 출력
    click.echo(
        click.style(
            "Output: ", fg="yellow"
        ) + command
    )

    # 명령어를 실행할 것인지 묻기
    click.echo(
        click.style(
            "명령어를 실행하시겠습니까? (y/n): ",
            fg="yellow"
        ),
        nl=False
    )

    # 사용자 선택 입력
    choice = input()

    # 사용자가 'y'를 입력하면 명령어 실행
    if choice == "y":
        os.system(command)
    elif choice == "n":
        continue
    else:
        click.echo(
            click.style(
                "잘못된 선택입니다. 'y' 또는 'n'을 입력해 주세요.",
                fg="red"
            )
        )

    click.echo()  # 빈 줄 추가


Input (종료하려면 'exit' 입력): 파일 검색
Output: find . -name "filename"
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): 빈 파일 하나 생성하기
Output: touch filename.txt
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): filename.txt 삭제하기
Output: rm filename.txt
명령어를 실행하시겠습니까? (y/n): y

Input (종료하려면 'exit' 입력): exit
